# Example: Markov chain Monte Carlo (MCMC)

In [ ]:
import sys
sys.path.append('../../../supplemental_material')

import numpy as np
import matplotlib.pyplot as plt
import synth_dataset as sd
import matplotlib.colors as mcolors
import emcee
import corner
from IPython.display import display, Math

# Define color-blind-friendly palette
hex1 = ['#648FFF', '#785EF0', '#DC267F', '#FE6100', '#FFB000']
hex1_inverted = hex1[::-1]
colors1=[mcolors.to_rgb(i) for i in hex1]

## Generate Dataset

In [18]:
# array of x values
x = np.linspace(0,1,300)
# produce the two gaussian peak signals
signal = sd.random_peaks(x,peaks_range=[1,2],c = 0.10, roi_position=[0.45,0.55],ph_min = 0.1, ph_max = 0.5, edge_tol=0.2,method=2)
# produce an exponential background 
background = sd.exponential_bg(len(x))
# produce threshold jump
threshold = sd.random_arctan_curve(len(x),itx_min=0.1,center_min=0.5,center_max=0.55,x_scale_min=0.05, x_scale_max=0.1)


In [19]:

# add random noise to generate the raw signal
snr = 35
raw = sd.add_noise(snr=snr, signal = signal + background + threshold)

In [ ]:
# plot the data
plt.scatter(x,raw,label='Raw data',color=colors1[0])
plt.plot(x,signal,label='Signal',ls='--',color=colors1[1])
plt.plot(x,background,label='Background',ls='--',color=colors1[2])
plt.plot(x,threshold,label='Threshold',ls='--',color=colors1[3])
plt.xlabel('x')
plt.ylabel('Intensity')
plt.legend()
plt.show()

## Define Theoretical Model

### Signals

In [21]:
def gaussians(x, params,ngauss):
    gaussians = []
    for i in range(ngauss):
        A=params['amp%s'%str(i)]
        mu=params['mu%s'%str(i)]
        sigma=params['sigma%s'%str(i)]
        val = A/np.sqrt(2*np.pi)/sigma * np.exp(-0.5 * ((x - mu) / sigma) ** 2)
        gaussians.append(val)
    return gaussians


### Threshold

In [22]:
def arctans(x, params,narctan):
    arctans = []
    for i in range(narctan):
        itx = params['itx%s'%str(i)]
        x_tan = params['x%s_tan'%str(i)]
        pos_tan = params['pos%s_tan'%str(i)]
        val = itx / np.pi * (np.arctan((x - pos_tan)/x_tan) + np.pi/2)
        arctans.append(val)
    return arctans

### Backgrounds

In [23]:

def backgrounds_poly(x,params,npoly):
    polys = []
    for i in range(npoly):
        poly = params['poly%s'%str(i)]
        val = poly*x**i
        polys.append(val)
    return polys

def backgrounds_exp(x,params):
    val = params['exp1']*np.exp(params['exp2']*x)
    return val


### Full Model

In [24]:
def raw_model(x,params,npoly,narctans,ngauss):
    model = np.sum(backgrounds_poly(x, params,npoly),axis=0) +backgrounds_exp(x,params)+ np.sum(arctans(x,params,narctans),axis=0) + np.sum(gaussians(x,params,ngauss),axis=0)
    return model

Adapt parameters dictionary to input expected by EMCEE

In [25]:
def parconverter(theta,npoly,narctans,ngauss):
    params = {'exp1':theta[0],'exp2':theta[1]}
    for i in range(npoly):
        params['poly%s'%str(i)] = theta[2+i]
    for i in range(narctans):
        params['itx%s'%str(i)] = theta[2+npoly+i]
        params['x%s_tan'%str(i)] = theta[2+npoly+narctans+i]
        params['pos%s_tan'%str(i)] = theta[2+npoly+2*narctans+i]
    for i in range(ngauss):
        params['amp%s'%str(i)] = theta[2+npoly+3*narctans+i]
        params['mu%s'%str(i)] = theta[2+npoly+3*narctans+ngauss+i]
        params['sigma%s'%str(i)] = theta[2+npoly+3*narctans+2*ngauss+i]
    return params

## Prior Distribution Setup

Given our knowledge of the experiment, and observation of data, we can provide a rough ansatz on the initial values of the parameters to make it easier for the sampler to find a good best fit model.

This procedure can, and should, be automatized. 

In [10]:
# select the degree of the polynomials
npoly = 0
# select the number of arctans we expect
narctans = 1
# select the number of gaussians we expect
ngauss = 1
                  
initial = np.array([1.0,-3.0,0.5,0.1,0.55,0.15,0.55,0.05]) #exp1,exp2,arc1,arc2,arc3,gauss1,gauss2,gauss3
params = parconverter(initial,npoly,narctans,ngauss)

In [ ]:
plt.scatter(x,raw,label='Raw', s=10, color=colors1[0])
plt.plot(x,raw_model(x,params,npoly,narctans,ngauss),label='Init Model',color=colors1[2])
#plt.plot(x,backgrounds_exp(x,params),label='Background Model',color=colors1[1])
#plt.plot(x,arctans(x,params,narctans)[0],label='Threshold Model',color=colors1[3])
#plt.plot(x,gaussians(x,params,ngauss)[0],label='Signal Model',color=colors1[4])
#plt.plot(x,signal,label='Signal',ls='--',color=colors1[4])
#plt.plot(x,background,label='Background',ls='--',color=colors1[1])
#plt.plot(x,threshold,label='Threshold',ls='--',color=colors1[3])
plt.xlabel('x')
plt.ylabel('Intensity')

plt.legend()

## EMCEE Setup

### Define Model

In [12]:
def model(theta,x,npoly,narctans,ngauss):
    params = parconverter(theta,npoly,narctans,ngauss)
    return raw_model(x,params,npoly,narctans,ngauss)

### Define Likelihood

In [13]:
def lnlike(theta, x, y, yerr, npoly, narctans, ngauss):
    ymodel = model(theta,x,npoly,narctans,ngauss)
    inv_sigma2 = 1.0/(yerr**2)
    return -0.5*(np.sum((y-ymodel)**2*inv_sigma2))

### Define Priors

In [14]:
def lnprior(theta, initial):
    j=0
    for i in range(len(theta)):
        if np.abs(theta[i]/initial[i]-1)>2.0:
            j+=1
    if j > 0:
        return -np.inf
    else:
        return 0.0

### Define Probability

In [15]:
    
def lnprob(theta, x, y, yerr, initial, npoly, narctans, ngauss):
    lp = lnprior(theta, initial)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, x, y, yerr, npoly, narctans, ngauss)


### Estimate Noise

In [16]:
squared_signal = signal ** 2
signal_power = np.sum(squared_signal) / len(squared_signal)
# Calculate the noise power using the specified SNR in units of dB
snr_linear = 10 ** (snr / 10)
# Calculate the standard deviation of the noise
noise_std_dev = np.sqrt(signal_power / snr_linear)

### Initialise emcee parameters

In [17]:

nwalkers = 2000
ndim = 2+npoly+3*narctans+3*ngauss
init = initial 
#p0 =np.random.rand(nwalkers, ndim)
p0 = np.random.normal(loc=initial, scale=1, size=(nwalkers, ndim))


### Initialise Sampler

In [18]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(x, raw, noise_std_dev, initial, npoly, narctans, ngauss))

### Burn-in Phase

In [ ]:
nsteps_burnin = 2000
state, _, _ = sampler.run_mcmc(p0, nsteps_burnin, progress=True)
sampler.reset()


### Production Phase

In [ ]:
nsteps = 2000
sampler.run_mcmc(state, nsteps,progress=True)
samples = sampler.chain[:, nsteps//10:, :].reshape((-1, ndim))    #skip the first 10% of the chain

### Results

In [29]:
def plotter(sampler):
    plt.ion()
    plt.plot(x,raw,label='raw')
    samples = sampler.flatchain
    for theta in samples[np.random.randint(len(samples), size=100)]:
        plt.plot(x, model(theta, x,npoly,narctans,ngauss), color="r", alpha=0.1)
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.xlabel('x')
    plt.ylabel('Intensity')
    plt.legend()
    plt.show()

In [ ]:
plotter(sampler)

In [31]:
samples = sampler.flatchain
theta_max = samples[np.argmax(sampler.flatlnprobability)]
params = parconverter(theta_max,npoly,narctans,ngauss)

In [ ]:
plt.scatter(x,raw,label='Raw', s=10, color=colors1[0])
plt.plot(x,raw_model(x,params,npoly,narctans,ngauss),label='Model',color=colors1[2])
plt.plot(x,signal+background+threshold,label='Truth',ls='--',color=colors1[2])
#plt.plot(x,backgrounds_poly(x,params,npoly)[0],label='Polynomial Background')
plt.plot(x,backgrounds_exp(x,params),label='Background Model',color=colors1[1])
plt.plot(x,arctans(x,params,narctans)[0],label='Threshold Model',color=colors1[3])
plt.plot(x,gaussians(x,params,ngauss)[0],label='Signal Model',color=colors1[4])
#plt.plot(x,gaussians(x,params,ngauss)[1],label='Gaussians')
plt.plot(x,signal,label='Signal',ls='--',color=colors1[4])
plt.plot(x,background,label='Background',ls='--',color=colors1[1])
plt.plot(x,threshold,label='Threshold',ls='--',color=colors1[3])
plt.xlabel('x')
plt.ylabel('Intensity')

plt.legend()

#plt.legend()

In [ ]:
flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)
labels = ['exp1','exp2','itx1','x1tan','pos1tan','amp1','mu1','sigma1']
fig = corner.corner(
    flat_samples, labels=labels
);

In [ ]:
for i in range(ndim):
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    txt = "\mathrm{{{3}}} = {0:.3f}_{{-{1:.3f}}}^{{{2:.3f}}}"
    txt = txt.format(mcmc[1], q[0], q[1], labels[i])
    display(Math(txt))